In [225]:
from groq import Groq
import os
import sys
import anthropic
import ollama
import random
import pandas as pd
from tqdm import tqdm
from google.generativeai.types import RequestOptions
from google.api_core import retry
from typing import List, Tuple
import json
from openai import OpenAI
import datetime
import openai
import time
import re

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from concurrent.futures import ThreadPoolExecutor, TimeoutError

# prompts

### Zero Shot - Vanilla CoT

In [226]:
zero_shot_vanilla_cot = """
Think through your answer step by step. Put the concise form of your final answer in curly brackets e.g. {A}, {True} or {3.0}.
"""

In [227]:
import re

def extract_last_sentence(text):
    # Split the text at the point where multiple-choice options start (e.g., "A)", "B)", etc.)
    parts = re.split(r'\n[A-Z]\)', text, flags=re.MULTILINE)
    
    if parts:
        # The first part contains everything before the options
        pre_options = parts[0]
        
        # Use a regex to find all sentences ending with ., ?, or !
        sentences = re.findall(r'[^.!?]*[.!?]', pre_options, re.DOTALL)
        
        if sentences:
            # Return the last sentence after stripping any extra spaces
            return sentences[-1].strip()
    
    # If splitting or finding sentences fails, return the original text
    return text

In [228]:
# import re

# def extract_last_sentence(text):
#     # Split the text into sentences, excluding option markers like "A)", "B)", etc.
#     sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|!)(?=\s[A-Z]\))', text)
    
#     # Return the last sentence after stripping any extra spaces.
#     return sentences[-1].strip() if sentences else text

In [229]:
def get_prompt(prompt_type: str, few_shot_prompt: str, question: str) -> str:
    # print(question)
    last_sentence = extract_last_sentence(question)
    
    # instruction = f"I want you to answer this question but your explanation should contain references referring back to the information in the question. To do that, first, re-generate the question with proper tags for key phrases, the key phrases that are most relevant to answering the question {last_sentence} and then generate your answers. The output format is as follow:\n\"
    
        # CHANGE FOR MATH
    instruction = f"I want you to answer this question but your explanation should contain references referring back to the information in the question. To do that, first, re-generate the question with proper tags for key phrases, the key phrases that are most relevant to answering the question and then generate your answers. The output format is as follow:\n\
                Reformatted Question: \
                    Answer:"
                    
    prompts = {
        "zero_shot_vanilla_cot": f"{question}\n{zero_shot_vanilla_cot}",
        "gcot": f"{few_shot_prompt}\n{question}\n{instruction}",
        "cot": f"{few_shot_prompt}\n{question}\nPlease generate your explanation first, then generate the final answer in the bracket as follow:\nAnswer: {{}}"
    }
    return prompts[prompt_type]

def save_results(save_path: str, ids: List[str], questions: List[str], answers: List[str], append: bool = False):
    df = pd.DataFrame({'id': ids, 'question': questions, 'answer': answers})
    if append and os.path.exists(save_path):
        df.to_csv(save_path, mode='a', index=False, header=False)
    else:
        df.to_csv(save_path, index=False)

def read_jsonl_file(filepath: str) -> List[dict]:
    data = []
    with open(filepath, 'r') as file:
        for line in file:
            json_obj = json.loads(line)
            data.append(json_obj)
    return data

def load_already_answered_ids(save_path: str) -> set:
    if os.path.exists(save_path):
        df = pd.read_csv(save_path)
        answered_ids = set(df['id'].tolist())
        # print(f"Loaded {len(answered_ids)} already answered IDs from: {save_path}")
        print(f"Already answered IDs: {answered_ids}")
        return answered_ids
    else:
        print(f"No existing save file found at: {save_path}. Starting fresh.")
        return set()

def initialize_save_file(save_path: str):
    if not os.path.exists(save_path):
        # Create an empty DataFrame with headers and save
        df = pd.DataFrame(columns=['id', 'question', 'answer'])
        df.to_csv(save_path, index=False)
        print(f"Initialized new save file with headers at: {save_path}")

def load_data_size_specific(data_path: str, sample_size: int = 0, random_seed: int = 0):
    random.seed(random_seed)

    if data_path.endswith('.jsonl'):
        data = read_jsonl_file(data_path)
    elif data_path.endswith('.json'):
        with open(data_path, 'r') as file:
            data = json.load(file)
    
    question_length = 0
    eligible_data = [x for x in data if len(x["question"]) >= question_length]
    
    if sample_size > 0 and sample_size < len(eligible_data):
        sampled_data = random.sample(eligible_data, sample_size)
    else:
        sampled_data = eligible_data
    
    ids = [x["id"] for x in sampled_data]
    questions = [x["question"] for x in sampled_data]
    
    return ids, questions

        
def get_longest_questions_and_ids(data_path, sample_size=200):
    data = read_jsonl_file(data_path)
    full_ids = [x["id"] for x in data]
    full_questions = [x["question"] for x in data]

    # Combine questions and IDs into a list of tuples
    full_questions_ids = list(zip(full_questions, full_ids))
    
    # Sort the tuples by the length of the questions
    sorted_full_questions_ids = sorted(full_questions_ids, key=lambda x: len(x[0]), reverse=True)
    
    # Select the shortest questions and their IDs
    sampled_data = random.sample(data, sample_size)
    longest_ = sorted_full_questions_ids[:min(sample_size, len(sorted_full_questions_ids))]

    # Separate them back into two lists
    longest_questions, longest_ids = zip(*longest_)

    # Convert to lists if necessary
    longest_questions = list(longest_questions)
    longest_ids = list(longest_ids)
    
    return longest_ids, longest_questions

def get_shortest_questions_and_ids(data_path, sample_size=200):
    data = read_jsonl_file(data_path)
    full_ids = [x["id"] for x in data]
    full_questions = [x["question"] for x in data]

    # Combine questions and IDs into a list of tuples
    full_questions_ids = list(zip(full_questions, full_ids))
    
    # Sort the tuples by the length of the questions in ascending order
    sorted_full_questions_ids = sorted(full_questions_ids, key=lambda x: len(x[0]))
    
    # Select the shortest questions and their IDs
    sampled_data = random.sample(data, sample_size)
    shortest_ = sorted_full_questions_ids[:min(sample_size, len(sorted_full_questions_ids))]

    # Separate them back into two lists
    shortest_questions, shortest_ids = zip(*shortest_)

    # Convert to lists if necessary
    shortest_questions = list(shortest_questions)
    shortest_ids = list(shortest_ids)
    
    return shortest_ids, shortest_questions

    
def get_random_questions_and_ids(data_path, sample_size=200):

    data = read_jsonl_file(data_path)

    longest_questions, longest_ids = get_longest_questions_and_ids(data_path, sample_size)
    
    result_ids = []
    result_questions = []
    # Select the shortest questions and their IDs
    sampled_data = random.sample(data, sample_size*2)
    totalQuestions = 0
    for x in sampled_data:
        if totalQuestions >= sample_size:
            break
        if x["question"] not in longest_questions:
            result_ids.append(x["id"])
            result_questions.append(x["question"])
            totalQuestions += 1
    
    return result_ids, result_questions

In [230]:
def query_4o(prompt: str) -> str:
    client = OpenAI()

    completion = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "user",
                "content": f"{prompt}"
            }
        ],
        temperature=0
    )

    return completion.choices[0].message.content

def query_llama(prompt):
    client = openai.OpenAI(
        api_key=os.environ.get("SAMBANOVA_API_KEY"),
        base_url="https://api.sambanova.ai/v1",
    )

    response = client.chat.completions.create(
        model='Meta-Llama-3.1-8B-Instruct',
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        temperature=0.6, # Meta default
        top_p = 0.9 # Meta default
    )
    time.sleep(2)  # Pause execution for 2 seconds
    return response.choices[0].message.content

def query_llama_70b(prompt):
    client = openai.OpenAI(
        api_key=os.environ.get("SAMBANOVA_API_KEY"),
        base_url="https://api.sambanova.ai/v1",
    )

    response = client.chat.completions.create(
        model='Meta-Llama-3.1-70B-Instruct',
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        temperature=0.6, # Meta default
        top_p = 0.9 # Meta default
    )
    time.sleep(2)  # Pause execution for 2 seconds
    return response.choices[0].message.content

def query_llama_405b(prompt):
    client = openai.OpenAI(
        api_key=os.environ.get("SAMBANOVA_API_KEY"),
        base_url="https://api.sambanova.ai/v1",
    )

    response = client.chat.completions.create(
        model='Meta-Llama-3.1-405B-Instruct',
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
        temperature=0.6, # Meta default
        top_p = 0.9 # Meta default
    )
    time.sleep(5)  # Pause execution for 2 seconds
    return response.choices[0].message.content

In [231]:
def query_llm(llm_model: str, ids: List[str], questions: List[str], few_shot_prompt: str, prompt_type: str, save_path: str, already_answered_ids: set) -> Tuple[List[str], List[str], List[str]]:
    answers = []
    ids_can_be_answered = []
    questions_can_be_answered = []
    
    for id, q in tqdm(zip(ids, questions), total=len(ids)):
        # print(q)
        # print(f"Processing ID: {id}")
        if id in already_answered_ids:
            print(f"Skipping: {id}", end=' ')
            continue
        # if id == 1146: # weird ID that breaks llama
        #     continue
        
        # print(few_shot_prompt)
        prompt = get_prompt(prompt_type, few_shot_prompt, q)
        # print(prompt)
        try:
            # print(prompt)
            if llm_model == 'gemini':
                answer = query_gemini(prompt, id)
            elif llm_model == 'claude':
                answer = query_claude(prompt)
            elif llm_model == '4o':
                # answer = query_4o_multiturn(prompt)
                if prompt_type == 'multi_convo':
                    fact_prompt = get_prompt(prompt_type="fact_prompt", few_shot_prompt="", question=q)
                    
                    answer_prompt = get_prompt(prompt_type="answer_prompt_data", few_shot_prompt="", question=q)
                    answer = query_4o_multiconvo(fact_prompt=fact_prompt, answer_prompt=answer_prompt, extracted_question=q)
                else:
                    answer = query_4o(prompt)
                
            elif llm_model == 'llama3.18b':
                answer = query_llama(prompt)
            elif llm_model == 'llama3.170b':
                answer = query_llama_70b(prompt)
            elif llm_model == 'llama3.1405b':
                answer = query_llama_405b(prompt)
            else:
                raise ValueError(f"Unsupported LLM model: {llm_model}")
            # print(f"Answer for ID {id}: {answer}")
            
            answers.append(answer)
            questions_can_be_answered.append(q)
            ids_can_be_answered.append(id)

            # Save after each answer
            save_results(save_path, [id], [q], [answer], append=True)
        except Exception as e:
            print(f"Error processing question {id}: {str(e)}")
            continue
    
    return ids_can_be_answered, questions_can_be_answered, answers

# Driver

In [232]:
json_datasets = ['logical_deduction_seven_objects','reasoning_about_colored_objects']
jsonl_datasets = ['GSM8K', 'date', 'GSM_Plus', 'MultiArith', 'ASDiv', 'SVAMP', 'AQUA', 'p_GSM8K', 'StrategyQA', 'commonsenseQA','SPARTQA']
all_datasets = jsonl_datasets + json_datasets

In [233]:
def run_model(llm_model, prompt_type, few_shot_txt, sample_size, project_root, identifier, isRandom = False, isLongest = False, isShortest = False):
    # for dataset in all_datasets:
    for dataset in ['MATH']:
        print(f"------Processing dataset: {dataset}-------")
        if few_shot_txt:
            fewshot_prompt_path = os.path.join(project_root, "prompt", dataset, few_shot_txt)
        # print(fewshot_prompt_path)
        # continue 
        if prompt_type == 'zero_shot_vanilla_cot':
            save_dir = os.path.join(project_root, 'logan/results/final/VanillaCoT', dataset, f'{llm_model}')
        elif prompt_type == 'gcot':
            save_dir = os.path.join(project_root, 'logan/results/final/GCoT', dataset, f'{llm_model}')
        elif prompt_type == 'cot':
            save_dir = os.path.join(project_root, 'logan/results/final/fewshot_CoT', dataset, f'{llm_model}')
        os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists
        save_path = os.path.join(save_dir, f'{prompt_type}_{identifier}_{few_shot_txt}_{dataset}_{llm_model}.csv')

        if dataset in json_datasets:
            data_path = os.path.join(project_root, 'data', dataset, 'test.json')
        else:
            data_path = os.path.join(project_root, 'data', dataset, 'test.jsonl')


        if isRandom:
            ids, questions = get_random_questions_and_ids(data_path, sample_size=sample_size)
        elif isLongest:
            ids, questions = get_longest_questions_and_ids(data_path, sample_size=sample_size)
        elif isShortest:
            ids, questions = load_data_size_specific(data_path, sample_size=sample_size)
        else:
            ids, questions = load_data_size_specific(data_path, sample_size=sample_size)
        print(sorted(ids))

        if few_shot_txt:
            with open(fewshot_prompt_path, 'r') as file:
                few_shot_prompt = file.read()
        else:
            few_shot_prompt = ""

        initialize_save_file(save_path)
        already_answered_ids = load_already_answered_ids(save_path)

        ids_answered, questions_answered, answers = query_llm(
            llm_model=llm_model,
            ids=ids,
            questions=questions,
            few_shot_prompt=few_shot_prompt,
            prompt_type=prompt_type,
            save_path=save_path,
            already_answered_ids=already_answered_ids
        )

        print(f"Processing complete for {dataset}. {len(ids_answered)} new answers saved to {save_path}.")

In [240]:
# llm_model = 'llama3.18b'
project_root = '/Users/log/Github/textual_grounding/'
prompt_type = 'cot'
few_shot_txt = 'cot_examples.txt'
sample_size = 200
isLongest = False 
isRandom = False
isShortest = False
identifier = 'random'

# models = ['llama3.18b']
models = ['llama3.170b']
for model in models:
    run_model(model, prompt_type, few_shot_txt, sample_size, project_root, 
              identifier=identifier, 
              isLongest=isLongest, 
              isRandom=isRandom,
              isShortest=isShortest)

------Processing dataset: MATH-------
['test/algebra/1031.json', 'test/algebra/1082.json', 'test/algebra/1098.json', 'test/algebra/1143.json', 'test/algebra/1184.json', 'test/algebra/1255.json', 'test/algebra/1265.json', 'test/algebra/1275.json', 'test/algebra/1282.json', 'test/algebra/1332.json', 'test/algebra/1339.json', 'test/algebra/1343.json', 'test/algebra/1457.json', 'test/algebra/1547.json', 'test/algebra/170.json', 'test/algebra/1796.json', 'test/algebra/1842.json', 'test/algebra/187.json', 'test/algebra/1934.json', 'test/algebra/1936.json', 'test/algebra/2036.json', 'test/algebra/2043.json', 'test/algebra/2046.json', 'test/algebra/2058.json', 'test/algebra/2064.json', 'test/algebra/2080.json', 'test/algebra/2176.json', 'test/algebra/2193.json', 'test/algebra/2214.json', 'test/algebra/224.json', 'test/algebra/2257.json', 'test/algebra/2264.json', 'test/algebra/2391.json', 'test/algebra/2470.json', 'test/algebra/2486.json', 'test/algebra/251.json', 'test/algebra/2517.json', 'te

100%|██████████| 200/200 [12:17<00:00,  3.69s/it]

Processing complete for MATH. 200 new answers saved to /Users/log/Github/textual_grounding/logan/results/final/fewshot_CoT/MATH/llama3.170b/cot_random_cot_examples.txt_MATH_llama3.170b.csv.


## Batch

In [ ]:
import os
import json
from pathlib import Path
from agents.batch_api_agents import prepare_batch_input, batch_api_agent
from openai import OpenAI
import importlib
import agents.batch_api_agents as batch_agents

# Reload the module to ensure the latest changes are loaded
importlib.reload(batch_agents)

llm_model = 'claude-3-5-sonnet-20240620'
project_root = '/Users/log/Github/textual_grounding/'
prompt_type = 'zero_shot_vanilla_cot'
few_shot_txt = None
sample_size = 2
json_datasets = ['logical_deduction_seven_objects','reasoning_about_colored_objects']
jsonl_datasets = ['GSM_Plus', 'MultiArith', 'SVAMP', 'p_GSM8K', 'StrategyQA', 'commonsenseQA','SPARTQA']
all_datasets = jsonl_datasets + json_datasets
all_datasets = ['MultiArith']
for dataset in all_datasets:
    if dataset in json_datasets:
        data_path = os.path.join(project_root, 'data', dataset, 'test.json')
    else:
        data_path = os.path.join(project_root, 'data', dataset, 'test.jsonl')
        
    save_dir = os.path.join(project_root, 'logan/results/final/VanillaCoT', dataset, f'{llm_model}')
    batch_dir = os.path.join(project_root, 'logan/batch_files/VanillaCoT', dataset, f'{llm_model}')
    os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists
    os.makedirs(batch_dir, exist_ok=True)  # Ensure the directory exists

    batch_results_path = os.path.join(save_dir, f'{prompt_type}_{few_shot_txt}_{dataset}_{llm_model}.jsonl')   
    batch_output_file = os.path.join(batch_dir, f'{prompt_type}_{few_shot_txt}_{dataset}_{llm_model}.jsonl')   

    ids, questions = load_data_size_specific(data_path, sample_size=sample_size)
    if few_shot_txt:
        with open(fewshot_prompt_path, 'r') as file:
            few_shot_prompt = file.read()
    else:
        few_shot_prompt = ""
    prompts = []
    for question in questions:
        prompt = get_prompt(prompt_type, few_shot_prompt, question)
        # print(prompt)
        prompts.append(prompt)

    # tasks = batch_agents.prepare_batch_input(
    #     llm_model=llm_model,
    #     ids=ids,
    #     prompts=prompts,
    #     batch_output_file=batch_output_file
    # )

    #Execute the batch processing with GPT-4 and save results
    batch_agents.batch_api_agent(
        llm_model=llm_model,
        ids=ids,
        prompts=prompts,
        batch_output_file=batch_output_file,
        batch_results_file=batch_results_path
    )
